# Notebook 02: Price Trends and Time-Varying Volatility

In [14]:
# --- SETUP (run only in Colab) ---
# Clone the repository to access project files
!git clone https://github.com/lautrevor/data-science-portfolio

fatal: destination path 'data-science-portfolio' already exists and is not an empty directory.


# VFV ETF Analysis

This notebook analyzes cleaned historical price data for VFV (Vanguard S&P 500 Index ETF – Canada).
The objective is to examine price trends, daily returns, and volatility in order to understand
risk and return characteristics of the ETF over time.

All calculations are time-aware and use only information available at time *t* to avoid
look-ahead bias.

## Data Loading and Preparation

In [15]:
import pandas as pd
import numpy as np
import altair as alt

alt.renderers.enable("default")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [22]:
vfv_df = pd.read_csv(
    "data-science-portfolio/project-2-vfv-signal-vs-noise/data/vfv_clean.csv"
)

# Drop the junk first row (the one with VFV.TO in Adj Close)
vfv_df = vfv_df.iloc[1:].reset_index(drop=True)

# Fix dtypes
vfv_df["Date"] = pd.to_datetime(vfv_df["Date"])
for c in ["Adj Close", "return", "target"]:
    vfv_df[c] = pd.to_numeric(vfv_df[c], errors="coerce")

vfv_df = vfv_df.dropna().reset_index(drop=True)

vfv_df.head()

,Date,Adj Close,return,target
0,2013-01-03,21.237473,0.005516,0.002352
1,2013-01-04,21.287413,0.002352,-0.001564
2,2013-01-07,21.254126,-0.001564,-0.003524
3,2013-01-08,21.179232,-0.003524,0.004322
4,2013-01-09,21.270769,0.004322,0.004695


> **Note:** When running this notebook in a hosted environment (e.g., Google Colab),
> the repository is cloned at runtime to ensure access to project files.

## Price Trend Analysis
### Long-Term Price Movement of VFV

This section visualizes the evolution of VFV’s adjusted closing price over time.
The objective is to identify long-term growth patterns and highlight periods of
market drawdowns and recovery.

In [23]:
price_chart = (
    alt.Chart(vfv_df)
    .mark_line()
    .encode(
        x=alt.X("Date:T", title="Date"),
        y=alt.Y("Adj Close:Q", title="Price (CAD)"),
        tooltip=["Date:T", "Close:Q"]
    )
    .properties(
        title="VFV Closing Price Over Time",
        width=700,
        height=400
    )
)

price_chart

alt.Chart(...)

The chart shows a clear long-term upward trend in VFV prices, consistent with
broad equity market growth. Short-term fluctuations reflect normal market
volatility and periods of increased uncertainty.

## Volatility Analysis
### Time-Varying Market Risk

Volatility measures the variability of returns over time and provides insight into
changing market risk. Rather than assuming constant risk, this section examines how
VFV’s volatility evolves using a rolling window approach.

In [24]:
#A 20-day rolling window approximates one trading month and allows volatility to be
#tracked as a time-varying quantity rather than a single fixed value.
vfv_df["rolling_volatility_20d"] = vfv_df["return"].rolling(20).std()
vfv_df[["Date", "rolling_volatility_20d"]].tail()

,Date,rolling_volatility_20d
3282,2026-02-02,0.007295
3283,2026-02-03,0.007317
3284,2026-02-04,0.007336
3285,2026-02-05,0.007548
3286,2026-02-06,0.008153


In [25]:
vol_chart = (
    alt.Chart(vfv_df)
    .mark_line()
    .encode(
        x=alt.X("Date:T", title="Date"),
        y=alt.Y(
            "rolling_volatility_20d:Q",
            title="20-Day Rolling Volatility"
        ),
        tooltip=["Date:T", "rolling_volatility_20d:Q"]
    )
    .properties(
        title="20-Day Rolling Volatility of VFV",
        width=700,
        height=400
    )
)

vol_chart

alt.Chart(...)

## Summary and Key Takeaways

This notebook analyzed VFV from a time-series perspective, focusing on long-term price
behavior and time-varying risk.

Key findings:
- VFV exhibits a strong long-term upward trend, consistent with broad equity market growth.
- Market risk is not constant over time; volatility clusters during periods of stress.
- The sharp volatility spike in 2020 highlights how sudden shocks can dramatically
  increase short-term risk.
- Rolling volatility provides more realistic insight into market risk than a single
  fixed volatility estimate.

Together, these results illustrate the importance of treating financial risk as a
dynamic quantity rather than a static assumption.